In [ ]:
#-*- coding:utf-8 -*-

import time
import csv
from bs4 import BeautifulSoup
import sys, io
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

# 앱 주소 크롤링
import lxml
import requests

import urllib.request
import time
import os


# Append your app store urls here
# 여기에 크롤링 할 앱의 주소 넣기
urls = []
path = os.getcwd()

# https://github.com/nahida-uap/Crawler-for-Google-Play-store를 참고함
# 참고한 소스 저자 @author Nahida Sultana Chowdhury <nschowdh@iu.edu>
#additionally add CSV file method here...

non_bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

# 크롬 드라이버 경로
# http://chromedriver.chromium.org/downloads 여기서 다운 받아서 같은 폴더에 넣어주기
chromedriver_loc = "./chromedriver"
driver = webdriver.Chrome(executable_path=chromedriver_loc)
wait = WebDriverWait( driver, 10 )


# 크롤링 할 게임들 추가하기
def get_link(target_url):
    
    response = requests.get(target_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    ytds = soup.find_all('a', {'class' : 'card-click-target'})
    
    for ytd in ytds:
        aria = ytd.get('aria-hidden', '')
        if(aria == 'true'):
            continue
        
        link = 'https://play.google.com' + ytd.get('href', '')
        print(link)
        urls.append(link)

# 리뷰 가져오기            
def get_review():
    
    for url in urls:
        #print(url)
        time.sleep(1)
        try:
            driver.get(url+'&hl=ko')
            driver.maximize_window()

            page = driver.page_source

            soup_expatistan = BeautifulSoup(page, "html.parser")
            expatistan_table = soup_expatistan.find("h1", class_="AHFaub")
            print("App name: ", expatistan_table.string)
            
            get_photo(url+'&hl=ko', expatistan_table.string)

            try:
                read_all_reviews_button = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div/div[6]/div/content')

                read_all_reviews_button.click()

                actions = ActionChains(driver)
                # 더 보기 버튼 나올때까지 
                for _ in range(15):
                    actions.send_keys(Keys.SPACE).perform()
                    time.sleep(1)
                try:
                    show_more_button = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div/div[2]/div[2]/div/content')

                    for i in range(0,5):
                        try:
                            show_more_button.click()
                # 이부분 range 안에 잇는 숫자를 늘리면 더 많이 크롤링 됐음
                # 50으로 했을때 약 5000개
                # 5일때 약 1500개
                            for _ in range(5):   #Adjust this range according to your need, I mean how far you wanna go down.
                                actions.send_keys(Keys.SPACE).perform()
                                time.sleep(1)
                        except Exception:
                            time.sleep(1)
                except Exception:
                    time.sleep(1)

                reviews_div = driver.find_element_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz[2]/div/div[2]/div/div[1]/div/div/div/div[2]/div').get_attribute("innerHTML")

                soup_expatistan = BeautifulSoup(reviews_div, "html.parser")

                expand_pages = soup_expatistan.find_all("div", class_="d15Mdf bAhLNe")

                try:
                    dir_path = path + '\\' + expatistan_table.string  ##진짜 app_name으로 변경하기

                    # dir 있나 확인
                    try:
                        if not(os.path.isdir(dir_path)):
                            os.mkdir(dir_path)
                    except OSError as e:
                        if e.errno != errno.EEXIST:
                            print("Failed to create directory!!!!!")
                            raise
                    myFile = open(dir_path + '\\' + 'review.csv',"w", newline='', encoding="utf-8")
                    with myFile:
                        writer = csv.writer(myFile)
                        for expand_page in expand_pages:
                            ind_rev = expand_page.find("span", class_="p2TkOb")
                            ind_rev_date = expand_page.find("div", class_="UD7Dzf")

                            num_of_star = 0
                            star_count = expand_page.find_all("div", class_="vQHuPe bUWb7c")    
                            for star in star_count:
                                num_of_star = num_of_star + 1
                            #print(num_of_star)

                            # 밑 3코드로 한글 문자 encode 하고 decode 해서 넣어줌
                            utf8_text = ind_rev.text.encode("utf-8")
                            utf8_date = ind_rev_date.text.encode("utf-8")
                            csvRow = [num_of_star, utf8_text.decode('utf-8'), utf8_date.decode('utf-8')]

                            writer.writerow(csvRow)
                    myFile.close()
                except Exception:
                    print("file open error")
            except Exception:
                time.sleep(1)
        except Exception:
                time.sleep(1)

# 게임 이미지 가져오는 함수 (앱 주소, 앱 이름)
def get_photo(target_url, app_name):
    dir_path = path + '\\' + app_name   ##진짜 app_name으로 변경하기
    print(dir_path)
    # dir 있나 확인
    try:
        if not(os.path.isdir(dir_path)):
            os.mkdir(dir_path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

    response = requests.get(target_url)    ##url로부터 response 끌고와 html소스 파싱
    soup = BeautifulSoup(response.text, "lxml")
    ytds = soup.find_all('button', {'class' : 'NIc6yf'})
  
    i = 0  #img_index
    
    for ytd in ytds :
        link = ytd.find('img')
        download_link = link.get('src', '')
        savename = str(i) + ".png"  # app name + str(i)로 변경 
        print(download_link)
        
        if(download_link == ''):
            continue
            
        urllib.request.urlretrieve(download_link, dir_path + '\\' + savename)
        
        print("saved")
        i = i + 1
        if (i == (len(ytds) / 3)):
            break  
                

# 크롤링 할 항목 (예를 들면 인기순위 주소) 넣어주기
# gl은 국가 코드 (한국에서의 랭킹이 필요하니까 KR 넣어줌)
target_url = 'https://play.google.com/store/apps/category/GAME/collection/topselling_free?gl=KR'
get_link(target_url)
get_review()

                
driver.quit()





In [ ]:
!pip install requests